In [8]:
import json
from yaml import safe_load
from typing import Final
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from functools import lru_cache

In [9]:
def read_client_data() -> dict[str, str]:
    with open('conf.yml', 'r') as f:
        data = safe_load(f)
        return data
    
client_data: dict[str, str] = read_client_data()
CLIENT_ID: Final[str] = client_data.get('client_id')
CLIENT_SECRET: Final[str] = client_data.get('client_secret')
REDIRECT_URI: Final[str] = client_data.get('redirect_uri')

In [10]:
# Authenticate with Spotify using OAuth
sp = Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                       client_secret=CLIENT_SECRET,
                                       redirect_uri=REDIRECT_URI,
                                       scope="user-library-read,user-top-read,user-read-recently-played"))

In [4]:
results = sp.current_user_top_tracks(time_range='short_term', limit=50)

In [ ]:
results['items'][0]['id']

In [4]:
@lru_cache(maxsize=1)
def get_saved_tracks():
    saved_tracks = []
    tracks = sp.current_user_saved_tracks()
    while tracks:
        for i, item in enumerate(tracks['items']):
           saved_tracks.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return saved_tracks


In [ ]:
saved_tracks = get_saved_tracks()

In [ ]:
for track in saved_tracks:
    print(track['track']['name'], end="\n")

In [49]:
top_tracks = sp.current_user_top_tracks(time_range='short_term')

In [ ]:
for item in top_tracks['items']:
    print(json.dumps(item))

In [12]:
@lru_cache(maxsize=1)
def get_recently_played(limit: int = 50, before: str = None, after: str = None):
    recently_played = []
    tracks = sp.current_user_recently_played(limit=limit, after=after, before=before)
    while tracks:
        for i, item in enumerate(tracks['items']):
           recently_played.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return recently_played

In [ ]:
before_unix_time = "1727038800"
recently_played = get_recently_played(after=before_unix_time)

In [ ]:
recently_played